<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Reservoir%20Simulation%20Ertekin/Unit%205%20Finite-Difference%20Approximation%20to%20Linear-Flow%20Equations/fd_linearflow_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 5 Exercises**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [16]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 941 (delta 15), reused 0 (delta 0), pack-reused 909
Receiving objects: 100% (941/941), 12.23 MiB | 39.52 MiB/s, done.
Resolving deltas: 100% (410/410), done.


## Exercise 5.5

<div>
<img src="https://user-images.githubusercontent.com/51282928/76333376-22151180-6324-11ea-82bc-b659ef6374b2.PNG" width="500"/>
</div>

### Explicit, timestep 10 days

In [10]:
"Task. determine pressure distribution during first year of production with timestep 10 days"

# known
p_initial = 4500 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 750
delta_z = 50
ngrid = 6
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 1.6E-06 # phase compressibility, psi^-1
k_x = 50 # perm in x direction, md
poro = 0.27
mu = 30 # phase viscosity, cp

delta_t = 10 # days

# conversion
k_x = k_x * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (alpha * B * delta_t) / (Vb * poro * c)
factor

3.46604938271605

In [11]:
# calculate transmissibility of coupling cells
beta = 1.127 # transmissibility conversion factor, is a constant
Ax = delta_y * delta_z
T_plus_half = beta * ((Ax * k_x) / (mu * B * delta_x))
T_min_half = T_plus_half
T_min_half, T_plus_half

(0.0704375, 0.0704375)

In [12]:
q = np.full(ngrid-1, T_min_half)
Ti_plus_halves = np.append(q, [0])
print(Ti_plus_halves)
p = np.full(ngrid-1, T_plus_half)
Ti_min_halves = np.append([0], p)
print(Ti_min_halves)
print("At grid 1, the coupling transmissibility coeffs are:", Ti_min_halves[0], "for T_min_half and:", Ti_plus_halves[0], "for T_plus_half.")
print("At grid 3, the coupling transmissibility coeffs are:", Ti_min_halves[2], "for T_min_half and:", Ti_plus_halves[2], "for T_plus_half.")
print("At grid 5, the coupling transmissibility coeffs are:", Ti_min_halves[4], "for T_min_half and:", Ti_plus_halves[4], "for T_plus_half.")

[0.0704375 0.0704375 0.0704375 0.0704375 0.0704375 0.       ]
[0.        0.0704375 0.0704375 0.0704375 0.0704375 0.0704375]
At grid 1, the coupling transmissibility coeffs are: 0.0 for T_min_half and: 0.0704375 for T_plus_half.
At grid 3, the coupling transmissibility coeffs are: 0.0704375 for T_min_half and: 0.0704375 for T_plus_half.
At grid 5, the coupling transmissibility coeffs are: 0.0704375 for T_min_half and: 0.0704375 for T_plus_half.


In [0]:
qsc = [40, -150, 0, 0, 100, 0] # production well in grid 4

In [14]:
pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

pi_arr = []
min_arr = []
plus_arr = []

for j in range(len(time)):
  pnew_arr = []

  minus = pi[0]
  plus = pi[-1]
  minus_arr = []
  plusus_arr = []
  for i, obj in enumerate(pi):
    if i > 0:
      minus = pi[i-1]
    if i < (len(pi) - 1):
      plus = pi[i+1]

    pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
    pnew_arr.append(float(pnew))

    minus_arr.append(float(minus))
    plusus_arr.append(float(plus))

  pi = pnew_arr

  min_arr.append(minus_arr)
  plus_arr.append(plusus_arr)
  pi_arr.append(pi)

df = pd.DataFrame.from_records(pi_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5', 'grid6'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5,grid6
0,15.0,4638.641975,3980.092593,4500.000000,4500.000000,4846.604938,4500.000000
1,25.0,4616.505801,3747.893453,4373.069882,4584.620079,5023.969719,4584.620079
2,35.0,4543.084885,3592.679419,4272.087234,4640.235001,5156.049144,4691.882836
3,45.0,4449.695009,3470.674387,4196.096252,4676.286249,5263.401798,4805.204330
4,55.0,4349.319033,3366.889318,4136.225379,4702.391037,5354.804170,4917.068593
5,65.0,4248.110762,3274.657751,4086.623387,4723.447483,5435.260362,5023.937292
6,75.0,4149.094062,3190.642190,4043.864357,4741.755234,5507.663054,5124.357646
7,85.0,4053.739738,3113.036617,4005.941799,4758.360880,5573.699243,5217.937772
8,95.0,3962.718591,3040.786072,3971.643543,4773.721984,5634.392040,5304.793326
9,105.0,3876.280096,2973.218541,3940.203449,4788.026532,5690.404935,5385.261508


### Implicit, timestep 10 days

In [15]:
# known
p_initial = 4500 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 750
delta_z = 50
ngrid = 6
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 1.6E-06 # phase compressibility, psi^-1
k_x = 50 # perm in x direction, md
poro = 0.27
mu = 30 # phase viscosity, cp

delta_t = 10 # days

# conversion
k_x = k_x * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (Vb * poro * c) / (alpha * B * delta_t)
factor

0.2885129118432769

In [17]:
import sys, os
sys.path.append('/content/reservoir-engineering/Reservoir Simulation Ertekin/Unit 5 Finite-Difference Approximation to Linear-Flow Equations')
from tdma_implicit import TDMAsolver

pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

p_new_arr = []

for j in range(len(time)):
  pi_next_plus_arr = []
  pi_next_arr = []
  pi_next_minus_arr = []
  pi_arr = []

  for i in range(len(pi)):
    pi_next_plus = Ti_plus_halves[i]
    pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
    pi_next_minus = Ti_min_halves[i]
    pi_current = -(qsc[i] + (factor * pi[i]))

    pi_next_plus_arr.append(float(pi_next_plus))
    pi_next_arr.append(float(pi_next))
    pi_next_minus_arr.append(float(pi_next_minus))
    pi_arr.append(float(pi_current))

  pi_next_plus_arr = pi_next_plus_arr[:-1]
  pi_next_minus_arr = pi_next_minus_arr[1:]

  p_new = TDMAsolver(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr, pi_arr)
  p_new_arr.append(p_new)
  pi = p_new # updating pi

df = pd.DataFrame.from_records(p_new_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5', 'grid 6'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5,grid 6
0,15.0,4542.570415,4149.060061,4447.642645,4531.768822,4746.020507,4548.277057
1,25.0,4527.801179,3899.426871,4378.099259,4571.921040,4930.206825,4623.223838
2,35.0,4479.162652,3712.059290,4307.043363,4610.981564,5074.912177,4711.859472
3,45.0,4411.036500,3564.111571,4240.738338,4645.778865,5193.349588,4806.343164
4,55.0,4332.317423,3442.003742,4181.082287,4675.808872,5293.538755,4901.946451
5,65.0,4248.542635,3337.520690,4128.082283,4701.555174,5380.485254,4995.851001
6,75.0,4163.168119,3245.593673,4081.032447,4723.754485,5457.405189,5086.422629
7,85.0,4078.344121,3163.024600,4039.048558,4743.105961,5526.427261,5172.765548
8,95.0,3995.392781,3087.743693,4001.290503,4760.179834,5589.003966,5254.444779
9,105.0,3915.106223,3018.372118,3967.038614,4775.408931,5646.157825,5331.311351


## Exercise 5.7

<div>
<img src="https://user-images.githubusercontent.com/51282928/76335943-da908480-6327-11ea-898b-4686f2d79e14.PNG" width="500"/>
</div>



## Example 5.12 Rework Exercise 5.8 With Varying Phase FVF 

![Example 5 8](https://user-images.githubusercontent.com/51282928/75949840-09e36380-5eda-11ea-824c-dd5c9ac6f6d5.PNG)

In [0]:
# known
p_initial = 6000 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 1000
delta_z = 75
ngrid = 5
grid_loc = 4 # grid location where production well is located
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 10 # days
qsc = -150 # minus means production, stb/d

# conversion
k_x = k_x * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (alpha * B * delta_t) / (Vb * poro * c)
factor